In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [ ]:
import requests
import pandas as pd

# Obter feriados
country = "BR"
year = 2024
response = requests.get(f"https://date.nager.at/api/v3/PublicHolidays/{year}/{country}")
holidays_data = response.json()

# Converter os dados de feriados em um DataFrame
holidays_df = pd.DataFrame(holidays_data)
holidays_df['date'] = pd.to_datetime(holidays_df['date'])  # Converter a coluna de datas

# Dicionários para tradução dos meses e dias da semana
month_translation = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

day_translation = {
    0: 'Segunda-feira', 1: 'Terça-feira', 2: 'Quarta-feira', 3: 'Quinta-feira',
    4: 'Sexta-feira', 5: 'Sábado', 6: 'Domingo'
}

# Adicionar colunas com nome do mês e do dia da semana em português
holidays_df['month'] = holidays_df['date'].dt.month
holidays_df['month_name'] = holidays_df['month'].map(month_translation)
holidays_df['weekday'] = holidays_df['date'].dt.dayofweek
holidays_df['weekday_name'] = holidays_df['weekday'].map(day_translation)

In [ ]:
import requests
import pandas as pd
import numpy as np
import json

# URL e parâmetros da API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": -22.9064,
    "longitude": -43.1822,
    "start_date": "2024-01-01",
    "end_date": "2024-08-01",
    "daily": ["weather_code", "temperature_2m_mean"],
    "timezone": "America/Sao_Paulo"
}

# Requisição à API Open-Meteo
response = requests.get(url, params=params)
data = response.json()

# Extraindo dados diários
daily_data = data['daily']
dates = pd.to_datetime(daily_data['time'])
weather_codes = daily_data['weather_code']
temperature_means = daily_data['temperature_2m_mean']

# Criar DataFrame
df = pd.DataFrame({
    'date': dates,
    'Código do Clima': weather_codes,
    'Temperatura Média': temperature_means
})

# Adicionar colunas de mês e ano
df['Ano'] = df['date'].dt.year
df['Mês'] = df['date'].dt.month

# Dicionário para tradução dos meses para português
month_translation = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

# Adicionar coluna com o nome do mês em português
df['Nome do Mês'] = df['Mês'].map(month_translation)

# Organizar colunas
df = df[['Ano','Mês' ,'Nome do Mês', 'Código do Clima', 'Temperatura Média','date']]

# Exibir o DataFrame resultante
print(df)


**1. Quantos feriados há no Brasil em 2024?**

In [ ]:
total_feriados = holidays_df.shape[0]
print(f"Total de feriados no Brasil em 2024: {total_feriados}")

**2. Qual mês de 2024 tem o maior número de feriados?**

In [ ]:
feriados_por_mes = holidays_df['month_name'].value_counts().reindex(month_translation.values())

print(feriados_por_mes)

Fevereiro, Março, Maio e Novembro, mas Fevereiro só tem o carnaval que conta como 2.

**3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [ ]:
holidays_df['weekday'] = holidays_df['date'].dt.weekday  # 0=Monday, 6=Sunday
feriados_dias_semana = holidays_df[holidays_df['weekday'] < 5].shape[0]
print(f"Feriados em dias de semana (segunda a sexta-feira): {feriados_dias_semana}")

**4. Qual foi a temperatura média em cada mês de 01/01/2024 a 01/08/2024 no Rio de Janeiro?**

In [ ]:
df_medias = pd.DataFrame(columns=['Mês', 'Nome do Mês','Temperatura Média'])

df_medias = df.groupby(['Mês','Nome do Mês'])['Temperatura Média'].mean().reset_index()

print(df_medias)

**5. Qual foi o tempo predominante em cada mês nesse período?**

In [ ]:
with open('/descriptions.json', 'r') as file:
    wmo_codes = json.load(file)

# Adicionar coluna com a descrição e imagem do clima
df['Descrição do Clima'] = df['Código do Clima'].map(lambda x: wmo_codes.get(str(x), {"day": {"description": "Desconhecido", "image": None}})['day']['description'])
df['Imagem do Clima'] = df['Código do Clima'].map(lambda x: wmo_codes.get(str(x), {"day": {"description": "Desconhecido", "image": None}})['day']['image'])

# Encontrar o clima predominante em cada mês
tempo_predominante = df.groupby(['Mês','Nome do Mês'])['Código do Clima'].agg(lambda x: x.mode().iloc[0]).reset_index()

# Adicionar a descrição e imagem do clima predominante
tempo_predominante['Descrição do Clima Predominante'] = tempo_predominante['Código do Clima'].map(lambda x: wmo_codes.get(str(x), {"day": {"description": "Desconhecido", "image": None}})['day']['description'])
tempo_predominante['Imagem do Clima Predominante'] = tempo_predominante['Código do Clima'].map(lambda x: wmo_codes.get(str(x), {"day": {"description": "Desconhecido", "image": None}})['day']['image'])

# Renomear colunas
tempo_predominante = tempo_predominante[['Nome do Mês', 'Descrição do Clima Predominante', 'Imagem do Clima Predominante']]

# Exibir o DataFrame resultante
print(tempo_predominante)

**6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [ ]:
df['Descrição do Clima'] = df['Código do Clima'].map(lambda x: wmo_codes.get(str(x), {"day": {"description": "Desconhecido", "image": None}})['day']['description'])
# Unir DataFrames de feriados e clima
df_holiday_temp = pd.merge(holidays_df, df[['date', 'Temperatura Média', 'Código do Clima','Descrição do Clima']], on='date', how='left')

# Filtrar de 1º de janeiro até 1º de agosto
start_date = '2024-01-01'
end_date = '2024-08-01'
df_holiday_temp = df_holiday_temp.loc[(df_holiday_temp['date'] >= start_date) & (df_holiday_temp['date'] <= end_date)]

# Selecionar colunas a serem exibidas
df_holiday_temp = df_holiday_temp[['localName', 'date', 'Temperatura Média', 'Código do Clima','Descrição do Clima']]

# Exibir o DataFrame resultante
df_holiday_temp.head(9)

In [ ]:
nao_aproveitavel = df_holiday_temp[
    (df_holiday_temp['Temperatura Média'] < 20) |
    (df_holiday_temp['Código do Clima'] > 2)
]
nao_aproveitavel.head(9)

**8. Qual foi o feriado "mais aproveitável" de 2024?**

In [ ]:
# Filtrar feriados que não são aproveitáveis
mais_aproveitavel = df_holiday_temp[
    (df_holiday_temp['Temperatura Média'] > 20) &
    (df_holiday_temp['Código do Clima'] <=2)
]

# Ordenar o DataFrame por temperatura média e código do clima
mais_aproveitavel_sorted = mais_aproveitavel.sort_values(by=['Temperatura Média', 'Código do Clima'], ascending=[False, True])

# Exibir os primeiros 5 resultados
mais_aproveitavel_sorted.head(5)


Ao organizar pela temperatura média e o Código do Clima, observamos que o feriado mais agradável de acordo com esses critérios seria o Carnaval. No entanto, se o critério priorizasse o tipo de clima, o melhor seria o Dia do Trabalhador. São muito próximos, então é interpretativo.